In [1]:
import pandas as pd

In [3]:
data = pd.read_csv(r"C:\Users\alext\Desktop\quantum_research\data\data_final_cleaned.csv",
                   on_bad_lines='skip')


In [4]:
data.head()

,Authors,Title,Year,Source title,Cited by,Affiliations,Authors with affiliations,Abstract,Author_list,Country
0,"Siverns J.D., Quraishi Q.",Ion trap architectures and new directions,2017,Quantum Information Processing,20,"Joint Quantum Institute, University of Marylan...","['Siverns J.D., Joint Quantum Institute, Unive...",Trapped ion technology has seen advances in pe...,"['Siverns J.D., Quraishi Q.']",['United States']
1,"Gupta S., Mittal S., Gupta T., Singhal I., Kha...",Parallel quantum-inspired evolutionary algorit...,2017,Applied Soft Computing Journal,29,"Shaheed Sukhdev College of Business Studies, U...","['Gupta S., Shaheed Sukhdev College of Busines...",The world around us may be viewed as a network...,"['Gupta S., Mittal S., Gupta T., Singhal I., K...","['India', 'UK']"
2,"Chouhan N., Saini H.K., Jain S.C.",A novel technique to modify the SHOR'S algorit...,2017,Proceedings of the 2017 2nd IEEE International...,4,"Dept. of Information Technology, M.L.V. Textil...","['Chouhan N., Dept. of Information Technology,...",This paper involves the study of traditional f...,"['Chouhan N., Saini H.K., Jain S.C.']",['India']
3,"Li M., Shen S., Jing N., Fei S.-M., Li-Jost X.",Tight upper bound for the maximal quantum valu...,2017,Physical Review A,21,"College of the Science, China University of Pe...","['Li M., College of the Science, China Univers...",It is a challenging task to detect genuine mul...,"['Li M., Shen S., Jing N., Fei S.-M., Li-Jost ...","['China', 'Germany']"
4,Rabindra B.,CloudGanga: Cloud computing based SDI model fo...,2017,International Journal of Agricultural and Envi...,14,"School of Computer Applications, KIIT Universi...","['Rabindra B., School of Computer Applications...",The present research paper proposes and develo...,['Rabindra B.'],['India']


In [5]:
labels = [
    "quantum optimization",
    "quantum algorithms",
    "quantum computing theory",
    "quantum resource theory",
    "quantum coherence",
    "quantum control",
    "quantum security tools",
    "quantum communication",
    "quantum cryptography",
    "quantum sensing",
    "quantum compilation",
    "quantum data structures",
    "quantum signal processing",
    "quantum distributed computing",
    "quantum simulation",
    "quantum error correction",
    "quantum software tools",
    "quantum computational complexity",
    "quantum error mitigation",
    "quantum state tomography",
    "quantum computing",
    "quantum surface codes",
    "quantum computing applications",
    "quantum hardware",
    "quantum thermodynamics",
    "quantum computing foundations",
    "quantum image processing",
    "quantum information theory",
    "quantum visualization tools",
    "quantum computing hardware (hybrid)",
    "quantum machine learning",
    "quantum workforce development",
    "quantum computing in ai",
    "quantum metrology",
    "quantum noise spectroscopy"
]

In [6]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time

In [7]:
API_KEY = "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"
MODEL = "anthropic/claude-3-haiku"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "HTTP-Referer": "https://openrouter.ai",
    "Content-Type": "application/json"
}


In [8]:
prompt_template = """
You are an expert in quantum computing.
Classify the following paper by choosing **exactly one** label from the list below.
Only output the label itself—no punctuation, quotes, or extra text.

Available labels:
{}

Title: {}
Abstract: {}
"""


In [10]:
ckpt = pd.read_csv("checkpoint_up_to_4000.csv")
import requests, json, time

In [ ]:
# 5) Now only classify those rows
for i in tqdm(indices, desc="Resuming from row 6001"):
    row = full.loc[i]
    prompt = prompt_template.format("\n".join(labels),
                                    row["Title"].strip(),
                                    row["Abstract"].strip())
    payload = {
      "model": MODEL,
      "messages": [{"role": "user", "content": prompt}]
    }
    try:
        resp   = requests.post("https://openrouter.ai/api/v1/chat/completions",
                               headers=headers, data=json.dumps(payload))
        label  = resp.json()["choices"][0]["message"]["content"].strip()
    except Exception:
        label = "error"

    # Optional clean‑up / fallback
    if label not in labels:
        label = "unknown"

    full.at[i, "Label"] = label
    time.sleep(0.2)

    # (optional) checkpoint more often if you like...

# 6) Save the updated full dataset
full.to_csv("abstracts_with_labels_full_resumed.csv", index=False)
print("✅ Done — all rows now labeled.")

In [12]:



predicted_labels = []
checkpoint_interval = 2000

for i, row in tqdm(data.iterrows(), total=len(data), desc="Classifying"):
    title    = row["Title"]
    abstract = row["Abstract"]

    prompt = prompt_template.format("\n".join(labels),
                                    title.strip(),
                                    abstract.strip())
    payload = {
      "model": MODEL,
      "messages": [{"role": "user", "content": prompt}]
    }

    try:
        resp   = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        result = resp.json()
        label  = result["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"Error at index {i}: {e}")
        label = "error"


    predicted_labels.append(label)

    #we will save the progress every 2000 rows for example in case the runtime disconnects
    #then we can just use the last saved labeled file and re-run the code again for any label = nan
    if (i + 1) % checkpoint_interval == 0:
        temp = data.iloc[: i+1].copy()
        temp["Label"] = predicted_labels
        temp.to_csv(f"checkpoint_up_to_{i+1}.csv", index=False)
        print(f"→ Saved checkpoint for rows 0–{i+1}")

    time.sleep(0.2)


data["Label"] = predicted_labels
data.to_csv("abstracts_with_labels_full.csv", index=False)
print("✅ Completed classification of all rows and saved final CSV.")



Classifying:   8%|████▊                                                         | 2000/25739 [41:01<6:20:36,  1.04it/s]

→ Saved checkpoint for rows 0–2000


Classifying:  12%|███████▎                                                    | 3126/25739 [1:04:12<6:37:35,  1.05s/it]

Error at index 3125: Expecting value: line 1 column 1 (char 0)


Classifying:  16%|█████████▎                                                  | 4000/25739 [1:22:30<6:22:57,  1.06s/it]

...

Error at index 23629: Expecting value: line 1 column 1 (char 0)


Classifying:  93%|████████████████████████████████████████████████████████▉    | 24000/25739 [9:55:44<46:13,  1.59s/it]

→ Saved checkpoint for rows 0–24000


Classifying: 100%|████████████████████████████████████████████████████████████| 25739/25739 [10:43:33<00:00,  1.50s/it]


✅ Completed classification of all rows and saved final CSV.


In [18]:
final_data = pd.read_csv(r"C:\Users\alext\Downloads\abstracts_with_labels_full.csv")


In [19]:
final_data.head(25)

,Authors,Title,Year,Source title,Cited by,Affiliations,Authors with affiliations,Abstract,Author_list,Country,Label
0,"Siverns J.D., Quraishi Q.",Ion trap architectures and new directions,2017,Quantum Information Processing,20,"Joint Quantum Institute, University of Marylan...","['Siverns J.D., Joint Quantum Institute, Unive...",Trapped ion technology has seen advances in pe...,"['Siverns J.D., Quraishi Q.']",['United States'],quantum computing
1,"Gupta S., Mittal S., Gupta T., Singhal I., Kha...",Parallel quantum-inspired evolutionary algorit...,2017,Applied Soft Computing Journal,29,"Shaheed Sukhdev College of Business Studies, U...","['Gupta S., Shaheed Sukhdev College of Busines...",The world around us may be viewed as a network...,"['Gupta S., Mittal S., Gupta T., Singhal I., K...","['India', 'UK']",quantum optimization
2,"Chouhan N., Saini H.K., Jain S.C.",A novel technique to modify the SHOR'S algorit...,2017,Proceedings of the 2017 2nd IEEE International...,4,"Dept. of Information Technology, M.L.V. Textil...","['Chouhan N., Dept. of Information Technology,...",This paper involves the study of traditional f...,"['Chouhan N., Saini H.K., Jain S.C.']",['India'],quantum algorithms
3,"Li M., Shen S., Jing N., Fei S.-M., Li-Jost X.",Tight upper bound for the maximal quantum valu...,2017,Physical Review A,21,"College of the Science, China University of Pe...","['Li M., College of the Science, China Univers...",It is a challenging task to detect genuine mul...,"['Li M., Shen S., Jing N., Fei S.-M., Li-Jost ...","['China', 'Germany']",quantum coherence
4,Rabindra B.,CloudGanga: Cloud computing based SDI model fo...,2017,International Journal of Agricultural and Envi...,14,"School of Computer Applications, KIIT Universi...","['Rabindra B., School of Computer Applications...",The present research paper proposes and develo...,['Rabindra B.'],['India'],quantum computing applications
5,"Farenick D., Rahaman M.",Bures contractive channels on operator algebras,2017,New York Journal of Mathematics,10,"Department of Mathematics & Statistics, Univer...","['Farenick D., Department of Mathematics & Sta...",In a unital C*-algebra with a faithful trace f...,"['Farenick D., Rahaman M.']",['Canada'],quantum computing theory
6,"Lusardi N., Geraci A., Gourgues R.B.M., Los J....",Array of superconducting nanowire single photo...,2017,"2016 IEEE Nuclear Science Symposium, Medical I...",4,"Politecnico di Milano, Department of Electroni...","['Lusardi N., Politecnico di Milano, Departmen...",Determining the precise number of photons in a...,"['Lusardi N., Geraci A., Gourgues R.B.M., Los ...","['Italy', 'Netherlands']",quantum sensing
7,Koloğlu M.,Quantum vacua of 2d maximally supersymmetric Y...,2017,Journal of High Energy Physics,3,Walter Burke Institute for Theoretical Physics...,"['Koloğlu M., Walter Burke Institute for Theor...",We analyze the classical and quantum vacua of ...,['Koloğlu M.'],['United States'],quantum computing theory
8,"Taherkhani E., Moaiyeri M.H., Angizi S.",Design of an ultra-efficient reversible full a...,2017,Optik,49,"Nanotechnology and Quantum Computing Lab, Shah...","['Taherkhani E., Nanotechnology and Quantum Co...",By the progressive scaling of the feature size...,"['Taherkhani E., Moaiyeri M.H., Angizi S.']","['United States', 'Iran']",quantum computing
9,Frolov A.V.,Can a quantum computer be applied for numerica...,2017,Russian Meteorology and Hydrology,17,Federal Service for Hydrometeorology and Envir...,"['Frolov A.V., Federal Service for Hydrometeor...",The paper considers fundamental limitations th...,['Frolov A.V.'],"['Russia', 'Russian Federation']",quantum computing


In [21]:
print(final_data["Label"].value_counts())

Label
quantum computing                                                                                                                                                                                           4707
quantum computing applications                                                                                                                                                                              2610
quantum optimization                                                                                                                                                                                        2035
quantum computing theory                                                                                                                                                                                    1971
quantum cryptography                                                                                                                                          

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Then print unique labels
print(final_data["Label"].unique())

['quantum computing' 'quantum optimization' 'quantum algorithms'
 'quantum coherence' 'quantum computing applications'
 'quantum computing theory' 'quantum sensing' 'quantum communication'
 'quantum computing hardware (hybrid)' 'quantum simulation'
 'quantum machine learning' 'quantum state tomography'
 'quantum contextuality' 'quantum emitters' 'quantum control'
 'quantum security tools' 'quantum image processing'
 'quantum information theory' 'quantum error correction'
 'quantum many-body' 'quantum phase transition' 'quantum hardware'
 'quantum topology' 'quantum imaging' 'quantum cryptography'
 'quantum thermodynamics' 'quantum anomalous Hall effect'
 'quantum computation' 'quantum signal processing' 'quantum memory'
 'quantum reflection' 'quantum computational complexity' 'quantum chaos'
 'quantum cosmology' 'quantum phase transitions'
 'This paper is not related to quantum computing. It is about atomic data of intermediate autoionizing Rydberg series of neutral argon atom. Therefo

In [24]:
valid_labels = [
    "quantum optimization",
    "quantum algorithms",
    "quantum computing theory",
    "quantum resource theory",
    "quantum coherence",
    "quantum control",
    "quantum security tools",
    "quantum communication",
    "quantum cryptography",
    "quantum sensing",
    "quantum compilation",
    "quantum data structures",
    "quantum signal processing",
    "quantum distributed computing",
    "quantum simulation",
    "quantum error correction",
    "quantum software tools",
    "quantum computational complexity",
    "quantum error mitigation",
    "quantum state tomography",
    "quantum computing",
    "quantum surface codes",
    "quantum computing applications",
    "quantum hardware",
    "quantum thermodynamics",
    "quantum computing foundations",
    "quantum image processing",
    "quantum information theory",
    "quantum visualization tools",
    "quantum computing hardware (hybrid)",
    "quantum machine learning",
    "quantum workforce development",
    "quantum computing in ai",
    "quantum metrology",
    "quantum noise spectroscopy",
    "quantum conversion",
    "quantum logic gates",
    "quantum reinforcement learning",
    "quantum fidelity estimation"
]

# Filter the DataFrame
filtered_df = final_data[final_data["Label"].isin(valid_labels)]

In [29]:
print(filtered_df["Label"].value_counts())

Label
quantum computing                      4707
quantum computing applications         2610
quantum optimization                   2035
quantum computing theory               1971
quantum cryptography                   1820
quantum algorithms                     1724
quantum error correction               1612
quantum machine learning               1284
quantum computing hardware (hybrid)    1020
quantum security tools                  783
quantum communication                   680
quantum coherence                       595
quantum error mitigation                553
quantum control                         551
quantum simulation                      514
quantum sensing                         438
quantum hardware                        420
quantum compilation                     195
quantum computing in ai                 194
quantum state tomography                175
quantum image processing                138
quantum information theory              138
quantum signal processing 

In [30]:
filtered_df.to_csv("filtered_abstracts.csv", index=False)

In [32]:
filtered_df = filtered_df.drop(["Source title"], axis = 1)

In [33]:
filtered_df.to_csv("cleaned_final_data.csv", index=False)